In [15]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import html5lib
import pandas as pd

def get_url(Departing, Arrival, Departure_date):
    TEMPLATES = "https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort={}&arrPort={}&departureDate={}%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en"
    url = TEMPLATES.format(Departing, Arrival, Departure_date)
    return url

# Prompt user for departure and arrival locations and departure date
a = input('''Choose your departure location from the following options:
ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg, 
KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
Enter your departure location (e.g., ABV for Abuja): ''')

b = input('''Choose your arrival location from the following options:
ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg,
KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
Enter your arrival location (e.g., LOS for Lagos): ''')

c = input('''DEPARTURE DATE
ENTER YOUR DEPARTURE DATE (e.g., dd.mm.yyyy): ''')

# Construct URL and print it
link = get_url(a, b, c)
print(link)

options = Options()
options.headless = True  # Run Chrome in headless mode

# Set up Chrome driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get(link)

sleep(2)

# Find flight rows
flight_rows = driver.find_elements(By.XPATH, '//div[@class="row w-100 no-gutters"]')
print(flight_rows)

# Initialize lists for storing flight details
depart_time_list = []
depart_loc_list = []
depart_date_list = []
flight_duration_list = []
total_stop_list = []
flight_no_list = []
arrival_time_list = []
arrival_loc_list = []
arrival_date_list = []
economy_price_list = []
executive_economy_price_list = []
business_price_list = []

# Parse the flight details
for WebElement in flight_rows:
    elementHTML = WebElement.get_attribute('outerHTML')
    elementsoup = BeautifulSoup(elementHTML, 'html.parser')

    # FLIGHT INFO BOX 1
    flight_box1 = elementsoup.find('div', {"class": "info-block"})
    depart_time = flight_box1.find("span", {"class": "time"})
    depart_loc = flight_box1.find("span", {"class": "port"})
    depart_date = flight_box1.find("span", {"class": "date"})
    depart_time_list.append(depart_time.text if depart_time else "N/A")
    depart_loc_list.append(depart_loc.text if depart_loc else "N/A")
    depart_date_list.append(depart_date.text if depart_date else "N/A")
    sleep(10)

    # FLIGHT INFO BOX 2
    flight_box2 = elementsoup.find('div', {"class": "info-row"})
    flight_duration = flight_box2.find("span", {"class": "flight-duration"})
    total_stop = flight_box2.find("span", {"class": "total-stop"})
    flight_no = flight_box2.find("span", {"class": "flight-no"})
    flight_duration_list.append(flight_duration.text if flight_duration else "N/A")
    total_stop_list.append(total_stop.text if total_stop else "N/A")
    flight_no_list.append(flight_no.text if flight_no else "N/A")
    sleep(10)

    # FLIGHT INFO BOX 3
    flight_box3 = elementsoup.find('div', {"class": "info-block text-right"})
    arrival_time = flight_box3.find("span", {"class": "time"})
    arrival_loc = flight_box3.find("span", {"class": "port"})
    arrival_date = flight_box3.find("span", {"class": "date"})
    arrival_time_list.append(arrival_time.text if arrival_time else "N/A")
    arrival_loc_list.append(arrival_loc.text if arrival_loc else "N/A")
    arrival_date_list.append(arrival_date.text if arrival_date else "N/A")
    sleep(10)

    # FLIGHT INFO BOX 4 (Money section)
    flight_box4 = elementsoup.find('div', {'class': "desktop-fare-block"})
    # Economy price
    economy_price = flight_box4.find("span", {"class": "currency-best-offer"})
    if economy_price is not None:
        economy_price_list.append(economy_price.text.strip())
    else:
        economy_price = flight_box4.find("span", {"class": "currency"})
        economy_price_list.append(economy_price.text.strip() if economy_price else "no seat")
    sleep(10)

    # Executive economy price
    executive_economy_price = flight_box4.find("span", {"class": "no-seat-text"})
    executive_economy_price_list.append(executive_economy_price.text.strip() if executive_economy_price else "no seat")

    # Business price
    business_price = flight_box4.find("span", {"class": "currency"})
    business_price_list.append(business_price.text.strip() if business_price else "no seat")
    sleep(10)

# Create a DataFrame with the collected data
Table = {
    'FLIGHT DEPART': depart_loc_list,
    'FLIGHT ARRIVAL': arrival_loc_list,
    'DEPART TIME': depart_time_list,
    'ARRIVAL TIME': arrival_time_list,
    'DEPART DATE': depart_date_list,
    'ARRIVAL DATE': arrival_date_list,
    'FLIGHT DURATION': flight_duration_list,
    'TOTAL STOP': total_stop_list,
    'FLIGHT NO': flight_no_list,
    'BUSINESS PRICE': business_price_list,
    'ECONOMY PRICE': economy_price_list,
    'EXECUTIVE ECONOMY PRICE': executive_economy_price_list
}
df = pd.DataFrame(Table)
print(df)


https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort=LOS&arrPort=ABV&departureDate=05.12.2024%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en
[<selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.106")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.107")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.108")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.109")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", 

In [16]:
df

,FLIGHT DEPART,FLIGHT ARRIVAL,DEPART TIME,ARRIVAL TIME,DEPART DATE,ARRIVAL DATE,FLIGHT DURATION,TOTAL STOP,FLIGHT NO,BUSINESS PRICE,ECONOMY PRICE,EXECUTIVE ECONOMY PRICE
0,Lagos (LOS),Abuja (ABV),06:30,07:50,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7120,$ 139.60,$ 55.60,No Seat
1,Lagos (LOS),Abuja (ABV),08:00,09:20,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7428,no seat,$ 55.60,No Seat
2,Lagos (LOS),Abuja (ABV),10:35,11:55,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7122,$ 139.60,$ 55.60,No Seat
3,Lagos (LOS),Abuja (ABV),12:00,13:20,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7124,$ 67.60,$ 67.60,No Seat
4,Lagos (LOS),Abuja (ABV),13:55,15:05,05 Dec 2024,05 Dec 2024,1h 10m,Nonstop,P4-7126,$ 67.60,$ 67.60,No Seat
5,Lagos (LOS),Abuja (ABV),15:45,17:00,05 Dec 2024,05 Dec 2024,1h 15m,Nonstop,P4-7132,$ 67.60,$ 67.60,No Seat
6,Lagos (LOS),Abuja (ABV),17:45,19:05,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7128,no seat,$ 55.60,No Seat
7,Lagos (LOS),Abuja (ABV),19:00,20:20,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7130,$ 139.60,$ 55.60,No Seat


In [14]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import html5lib

def get_url(Departing, Arrival, Departure_date):
    TEMPLATES = "https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort={}&arrPort={}&departureDate={}%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en"
    url = TEMPLATES.format(Departing, Arrival, Departure_date)
    return url

try:
    # Prompt user for departure and arrival locations and departure date
    a = input('''Choose your departure location from the following options:
    ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
    DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg, 
    KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
    QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
    Enter your departure location (e.g., ABV for Abuja): ''')

    b = input('''Choose your arrival location from the following options:
    ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
    DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg,
    KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
    QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
    Enter your arrival location (e.g., LOS for Lagos): ''')

    c = input('''DEPARTURE DATE
    ENTER YOUR DEPARTURE DATE (e.g., dd.mm.yyyy): ''')

    # Construct URL and print it
    link = get_url(a, b, c)
    print(link)

    options = Options()
    options.headless = True  # Run Chrome in headless mode

    # Set up Chrome driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(link)

    # Wait for the page to load
    time.sleep(5)

    # Extract flight rows
    flight_rows = driver.find_elements(By.XPATH, '//div[@class="row w-100 no-gutters"]')

    # Print flight information
    for row in flight_rows:
        print(row.text)
    
finally:
    driver.quit()


https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort=LOS&arrPort=ABV&departureDate=05.12.2024%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en
06:30
Lagos (LOS)
05 Dec 2024
1h 20m
Nonstop
P4-7120
07:50
Abuja (ABV)
05 Dec 2024
Flight Info
BEST OFFER
From
$ 55.60
From
$ 139.60
No Seat
08:00
Lagos (LOS)
05 Dec 2024
1h 20m
Nonstop
P4-7428
09:20
Abuja (ABV)
05 Dec 2024
Flight Info
BEST OFFER
From
$ 55.60
No Seat
No Seat
10:35
Lagos (LOS)
05 Dec 2024
1h 20m
Nonstop
P4-7122
11:55
Abuja (ABV)
05 Dec 2024
Flight Info
BEST OFFER
From
$ 55.60
From
$ 139.60
No Seat
12:00
Lagos (LOS)
05 Dec 2024
1h 20m
Nonstop
P4-7124
13:20
Abuja (ABV)
05 Dec 2024
Flight Info
From
$ 67.60
From
$ 139.60
No Seat
13:55
Lagos (LOS)
05 Dec 2024
1h 10m
Nonstop
P4-7126
15:05
Abuja (ABV)
05 Dec 2024
Flight Info
From
$ 67.60
From
$ 139.60
No Seat
15:45
Lagos (LOS)
05 Dec 2024
1h 15m
Nonstop
P4-7132
17:00
Abuja (ABV)
05 Dec 2024
Flight Info
From
$ 67.60
No Seat
No Seat
17:45
Lagos (LOS)
05 Dec 2